## Main code for Kaggle - Optiver Realized Volatility Prediction
@LaurentMombaerts 

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## MACHINE TO SET UP

In [2]:
###########################
machine = 'local'
###########################

**Lib Import / Data loading**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import time

# ML
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Maths
from scipy.interpolate import interp1d
# from arch import arch_model

# Paths tricks
import os
from pathlib import Path

# Support code
from support_file import *
from information_measures import *

if machine == 'local':
    datapath = os.path.join(str(Path.home()), 'ownCloud', 'Data', 'Kaggle', 'optiver-realized-volatility-prediction')

    # Load dataset
    train = pd.read_csv(os.path.join(datapath,'train.csv')) 
    all_stocks_ids = train['stock_id'].unique()
    all_time_ids = train['time_id'].unique()

    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    train = train[['row_id','target']]

    # Load test ids
    test = pd.read_csv(os.path.join(datapath,'test.csv'))
    test = test.drop(['stock_id','time_id'],axis=1)
    
elif machine == 'kaggle':
    
    # Load dataset
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    all_stocks_ids = train['stock_id'].unique()
    all_time_ids = train['time_id'].unique()

    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    train = train[['row_id','target']]

    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv') 
    test = test.drop(['stock_id','time_id'],axis=1)
    
    datapath = 0
    

**Functions**

In [4]:
# Competition metric
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

# Prediction function (chose here which prediction strategy to use)
def prediction_function(pred, machine, targets, all_stocks_ids, datapath):
        
    if pred == 'entropy':
        if machine == 'local':
            # Load data
            df_features_encoded_test = computeFeatures_wEntropy(machine=machine, dataset='test', all_stocks_ids=all_stocks_ids, datapath=datapath)
            df_features_encoded_train = computeFeatures_wEntropy(machine=machine, dataset='train', all_stocks_ids=all_stocks_ids, datapath=datapath)
            X = df_features_encoded_train.drop(['row_id'],axis=1)
            y = targets
            
            # Model
            model = CatBoostRegressor(verbose=0)
            model.fit(X,y)
            
            # Predicting targets from same
            yhat = model.predict(X)
            
            print('New model catboost perf : ', rmspe(y, yhat))
            
            # Submission file
            yhat_pd = pd.DataFrame(yhat,columns=['target'])
            return pd.concat([df_features_encoded_train['row_id'],yhat_pd],axis=1)

        # Features computation
        df_features_encoded_test = computeFeatures_wEntropy(machine=machine, dataset='test', all_stocks_ids=all_stocks_ids, datapath=datapath)
        df_features_encoded_train = computeFeatures_wEntropy(machine=machine, dataset='train', all_stocks_ids=all_stocks_ids, datapath=datapath)
        
        # Training model
        X = df_features_encoded_train.drop(['row_id'],axis=1)
        y = targets
        
        # Optimized model
        model = CatBoostRegressor(verbose=0)
        model.fit(X,y)
        
        # Predicting targets from test
        X_test = df_features_encoded_test.drop(['row_id'],axis=1)
        yhat = model.predict(X_test)
        
        # Submission file
        yhat_pd = pd.DataFrame(yhat,columns=['target'])
        submission_file = pd.concat([df_features_encoded_test['row_id'],yhat_pd],axis=1)
        
    if pred == 'new_test_laurent':
        if machine == 'local':
            # Load data
            df_features_encoded_test = computeFeatures_newTest_Laurent(machine=machine, dataset='test', all_stocks_ids=all_stocks_ids, datapath=datapath)
            df_features_encoded_train = computeFeatures_newTest_Laurent(machine=machine, dataset='train', all_stocks_ids=all_stocks_ids, datapath=datapath)
            X = df_features_encoded_train.drop(['row_id'],axis=1)
            y = targets
            
            # Model
            model = CatBoostRegressor(verbose=0)
            model.fit(X,y)
            
            # Predicting targets from same
            yhat = model.predict(X)
            
            print('New model catboost perf : ', rmspe(y, yhat))
            
            # Submission file
            yhat_pd = pd.DataFrame(yhat,columns=['target'])
            return pd.concat([df_features_encoded_test['row_id'],yhat_pd],axis=1)

        # Features computation
        df_features_encoded_test = computeFeatures_newTest_Laurent(machine=machine, dataset='test', all_stocks_ids=all_stocks_ids, datapath=datapath)
        df_features_encoded_train = computeFeatures_newTest_Laurent(machine=machine, dataset='train', all_stocks_ids=all_stocks_ids, datapath=datapath)
        
        # Training model
        X = df_features_encoded_train.drop(['row_id'],axis=1)
        y = targets
        
        # Optimized model
        model = CatBoostRegressor(verbose=0)
        model.fit(X,y)
        
        # Predicting targets from test
        X_test = df_features_encoded_test.drop(['row_id'],axis=1)
        yhat = model.predict(X_test)
        
        # Submission file
        yhat_pd = pd.DataFrame(yhat,columns=['target'])
        submission_file = pd.concat([df_features_encoded_test['row_id'],yhat_pd],axis=1)
        
    if pred == 'new_test_laurent_withoutEncoding':
        if machine == 'local':
            # Load data
            df_features_encoded_test = computeFeatures_newTest_Laurent_noCode(machine=machine, dataset='test', all_stocks_ids=all_stocks_ids, datapath=datapath)
            df_features_encoded_train = computeFeatures_newTest_Laurent_noCode(machine=machine, dataset='train', all_stocks_ids=all_stocks_ids, datapath=datapath)
            X = df_features_encoded_train.drop(['row_id'],axis=1)
            y = targets
            
            # Model
            model = CatBoostRegressor(verbose=0)
            model.fit(X,y)
            
            # Predicting targets from same
            yhat = model.predict(X)
            
            print('New model catboost perf : ', rmspe(y, yhat))
            
            # Submission file
            yhat_pd = pd.DataFrame(yhat,columns=['target'])
            return pd.concat([df_features_encoded_test['row_id'],yhat_pd],axis=1)

        # Features computation
        df_features_encoded_test = computeFeatures_newTest_Laurent_noCode(machine=machine, dataset='test', all_stocks_ids=all_stocks_ids, datapath=datapath)
        df_features_encoded_train = computeFeatures_newTest_Laurent_noCode(machine=machine, dataset='train', all_stocks_ids=all_stocks_ids, datapath=datapath)
        
        # Training model
        X = df_features_encoded_train.drop(['row_id'],axis=1)
        y = targets
        
        # Optimized model
        model = CatBoostRegressor(verbose=0)
        model.fit(X,y)
        
        # Predicting targets from test
        X_test = df_features_encoded_test.drop(['row_id'],axis=1)
        yhat = model.predict(X_test)
        
        # Submission file
        yhat_pd = pd.DataFrame(yhat,columns=['target'])
        submission_file = pd.concat([df_features_encoded_test['row_id'],yhat_pd],axis=1)
        
    if pred == 'new_test_laurent_withoutEncoding_wTrades':
        if machine == 'local':
            # Load data
            df_features_encoded_test = computeFeatures_newTest_Laurent_wTrades(machine=machine, dataset='test', all_stocks_ids=all_stocks_ids, datapath=datapath)
            df_features_encoded_train = computeFeatures_newTest_Laurent_wTrades(machine=machine, dataset='train', all_stocks_ids=all_stocks_ids, datapath=datapath)
            X = df_features_encoded_train.drop(['row_id'],axis=1)
            y = targets
            
            # Model
            model = CatBoostRegressor(verbose=0)
            model.fit(X,y)
            
            # Predicting targets from same
            yhat = model.predict(X)
            
            print('New model catboost perf : ', rmspe(y, yhat))
            
            # Submission file
            yhat_pd = pd.DataFrame(yhat,columns=['target'])
            return pd.concat([df_features_encoded_test['row_id'],yhat_pd],axis=1)

        # Features computation
        df_features_encoded_test = computeFeatures_newTest_Laurent_wTrades(machine=machine, dataset='test', all_stocks_ids=all_stocks_ids, datapath=datapath)
        df_features_encoded_train = computeFeatures_newTest_Laurent_wTrades(machine=machine, dataset='train', all_stocks_ids=all_stocks_ids, datapath=datapath)
        
        # Training model
        X = df_features_encoded_train.drop(['row_id'],axis=1)
        y = targets
        
        # Optimized model
        model = CatBoostRegressor(verbose=0)
        model.fit(X,y)
        
        # Predicting targets from test
        X_test = df_features_encoded_test.drop(['row_id'],axis=1)
        yhat = model.predict(X_test)
        
        # Submission file
        yhat_pd = pd.DataFrame(yhat,columns=['target'])
        submission_file = pd.concat([df_features_encoded_test['row_id'],yhat_pd],axis=1)
        
        
    if pred == 'garch':
        
        if machine == 'local':
            book_path_train = glob.glob(os.path.join(datapath,'book_train.parquet','*')) 
            
            # fit garch and predict
            prediction = garch_volatility_per_stock(list_file=book_path_train, prediction_column_name='pred')
            
            # Merge and evaluate results
            prediction = train.merge(prediction[['row_id','pred']], on = ['row_id'], how = 'left')
            prediction = prediction[prediction.pred.notnull()]

            # Estimate performances
            R2 = round(r2_score(y_true = prediction['target'], y_pred = prediction['pred']),3)
            RMSPE = round(rmspe(y_true = prediction['target'], y_pred = prediction['pred']),3)

            print('--')
            print(f'Performance of prediction: R2 score: {R2}, RMSPE: {RMSPE}')

            prediction = prediction.drop(columns=['target'])
            prediction = prediction.rename(columns={'pred': 'target'})
            
            return prediction
        
    if pred == 'new_version':
        if machine == 'local':
            # Load data
            df_features_encoded_test = computeFeatures_july(machine=machine, dataset='test', all_stocks_ids=all_stocks_ids, datapath=datapath)
            df_features_encoded_train = computeFeatures_july(machine=machine, dataset='train', all_stocks_ids=all_stocks_ids, datapath=datapath)
            X = df_features_encoded_train.drop(['row_id'],axis=1)
            y = targets
            
            # Model
            model = CatBoostRegressor(verbose=0)
            model.fit(X,y)
            
            # Predicting targets from same
            yhat = model.predict(X)
            
            print('New model catboost perf : ', rmspe(y, yhat))
            
            # Submission file
            yhat_pd = pd.DataFrame(yhat,columns=['target'])
            return pd.concat([df_features_encoded_train['row_id'],yhat_pd],axis=1)

        # Features computation
        df_features_encoded_test = computeFeatures_july(machine=machine, dataset='test', all_stocks_ids=all_stocks_ids, datapath=datapath)
        df_features_encoded_train = computeFeatures_july(machine=machine, dataset='train', all_stocks_ids=all_stocks_ids, datapath=datapath)
        
        # Training model
        X = df_features_encoded_train.drop(['row_id'],axis=1)
        y = targets
        
        # Optimized model
        model = CatBoostRegressor(verbose=0)
        model.fit(X,y)
        
        # Predicting targets from test
        X_test = df_features_encoded_test.drop(['row_id'],axis=1)
        yhat = model.predict(X_test)
        
        # Submission file
        yhat_pd = pd.DataFrame(yhat,columns=['target'])
        submission_file = pd.concat([df_features_encoded_test['row_id'],yhat_pd],axis=1)
        

    return submission_file

**Submission**

In [4]:
# New sub
import warnings
warnings.filterwarnings('ignore')
df_submission = prediction_function(pred='new_test_laurent_withoutEncoding_wTrades',machine=machine,targets=train['target'],all_stocks_ids=all_stocks_ids, datapath=datapath)
df_submission.to_csv('submission.csv',index=False)

Computing one stock took 0.7710602283477783 seconds for stock  0
Computing one stock took 21.529297351837158 seconds for stock  0
Computing one stock took 22.306584119796753 seconds for stock  1
Computing one stock took 22.503216981887817 seconds for stock  2
Computing one stock took 22.554168939590454 seconds for stock  3
Computing one stock took 22.087051153182983 seconds for stock  4
Computing one stock took 22.01963186264038 seconds for stock  5
Computing one stock took 22.310144186019897 seconds for stock  6
Computing one stock took 21.86559820175171 seconds for stock  7
Computing one stock took 22.24520492553711 seconds for stock  8
Computing one stock took 22.276533365249634 seconds for stock  9
Computing one stock took 23.402235746383667 seconds for stock  10
Computing one stock took 22.36283278465271 seconds for stock  11
Computing one stock took 22.92237424850464 seconds for stock  13
Computing one stock took 23.051005125045776 seconds for stock  14
Computing one stock took 2

In [5]:
import warnings
warnings.filterwarnings('ignore')
df_submission = prediction_function(pred='new_test_laurent_withoutEncoding_wTrades',machine=machine,targets=train['target'],all_stocks_ids=all_stocks_ids, datapath=datapath)
df_submission.to_csv('submission.csv',index=False)

Computing one stock took 0.8217861652374268 seconds for stock  0
Computing one stock took 36.58018517494202 seconds for stock  0
Computing one stock took 37.26157021522522 seconds for stock  1
Computing one stock took 37.62245202064514 seconds for stock  2
Computing one stock took 37.35794997215271 seconds for stock  3
Computing one stock took 36.98911905288696 seconds for stock  4
Computing one stock took 37.34412598609924 seconds for stock  5
Computing one stock took 37.54107093811035 seconds for stock  6
Computing one stock took 37.58751177787781 seconds for stock  7
Computing one stock took 38.015620946884155 seconds for stock  8
Computing one stock took 36.760937213897705 seconds for stock  9
Computing one stock took 37.910584926605225 seconds for stock  10
Computing one stock took 38.25134634971619 seconds for stock  11
Computing one stock took 38.303149938583374 seconds for stock  13
Computing one stock took 38.78213286399841 seconds for stock  14
Computing one stock took 38.325